In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import statsmodels.formula.api as smf 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # logistic regression
import statsmodels.formula.api as smf                # logistic regression
from sklearn.metrics import confusion_matrix         # confusion matrix
from sklearn.metrics import roc_auc_score            # auc score
from sklearn.neighbors import KNeighborsClassifier   # KNN for classification
from sklearn.neighbors import KNeighborsRegressor    # KNN for regression
from sklearn.preprocessing import StandardScaler     # standard scaler
from sklearn.tree import DecisionTreeClassifier      # classification trees
from sklearn.tree import plot_tree  
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer   
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#importing the excel file under new name and reading it
#file = './GOT_character_predictions.xlsx'
#got = pd.read_excel(io=file)

#displaying the first ten rows
#got.head(n=10)

In [3]:
#dropping index since it will be considered a feature by the models
#got = got.drop('S.No', axis=1)

In [4]:
#flaggin missing values

# creating a loop to identify the features with missing values and create a column with m_ identificator
# marking a 0 if value is not missing and 1 if the value is missing
#for col in got:
#    if got[col].isnull().astype(int).sum(axis=0) > 0:
#        got['m_'+col] = got[col].isnull().astype(int)
        
#checking loop works properly by summing values in the new columns 
#got.iloc[:,24:37].sum(axis=0)

Given the high number of missing values for this features and the lack of knowledge about the data, there will not be an imputation with mean or mode. 

In [5]:
# splitting the name column to obtain only first names
# setting a placeholder list
#namesholder_lst = []

# looping over each name
#for index, col in got.iterrows():
    
    # splitting name after first space
#    split_name = got.loc[index, 'name'].split(' ',1)
    
    # appending nameholder_lst with the results
#    namesholder_lst.append(split_name)
    

# converting nameholder_lst into a DataFrame 
#names_sep_df = pd.DataFrame(namesholder_lst)


# showing results
#names_sep_df

In [6]:
# renaming column to concatenate
#names_sep_df.columns = ["first name" , "1"]


# concatenating personal_email_domain with friends DataFrame
#got = pd.concat([got, names_sep_df['first name']],
#                   axis = 1)

#got

In [7]:
# guessing gender based on first name

# placeholder list
#names_guess_lst = []


# looping to guess gender
#for name in got['first name']:
#    guess = gender.Detector().get_gender(name)
#    print(guess)
#    names_guess_lst.append(guess)


# converting list into a series
#got['gender_guess'] = pd.Series(names_guess_lst)


# checking results
#got

In [8]:
# exporting to excel to not loose the gender_guesser data 

#got.to_excel('./got_copy.xlsx',
#                           index = False)

In [9]:
#loading file exported to excel previously 
file_new = './got_copy.xlsx'
got_c = pd.read_excel(io=file_new)
got_c.head(n=10)

,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,book1_A_Game_Of_Thrones,...,m_heir,m_house,m_spouse,m_isAliveMother,m_isAliveFather,m_isAliveHeir,m_isAliveSpouse,m_age,first name,gender_guess
0,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,NaN,NaN,0,...,0,1,1,0,0,0,1,1,Viserys,unknown
1,Walder Frey,Lord of the Crossing,Rivermen,208.0,NaN,NaN,NaN,House Frey,Perra Royce,1,...,1,0,0,1,1,1,0,0,Walder,unknown
2,Addison Hill,Ser,NaN,NaN,NaN,NaN,NaN,House Swyft,NaN,0,...,1,0,1,1,1,1,1,1,Addison,andy
3,Aemma Arryn,Queen,NaN,82.0,NaN,NaN,NaN,House Arryn,Viserys I Targaryen,0,...,1,0,0,1,1,1,0,0,Aemma,unknown
4,Sylva Santagar,Greenstone,Dornish,276.0,NaN,NaN,NaN,House Santagar,Eldon Estermont,0,...,1,0,0,1,1,1,0,0,Sylva,female
5,Tommen Baratheon,NaN,NaN,NaN,Cersei Lannister,Robert Baratheon,Myrcella Baratheon,NaN,NaN,0,...,0,1,1,0,0,0,1,1,Tommen,unknown
6,Valarr Targaryen,Hand of the King,Valyrian,183.0,NaN,NaN,NaN,House Targaryen,Kiera of Tyrosh,0,...,1,0,0,1,1,1,0,0,Valarr,unknown
7,Viserys I Targaryen,NaN,NaN,NaN,Alyssa Targaryen,Baelon Targaryen,Rhaenyra Targaryen,NaN,NaN,0,...,0,1,1,0,0,0,1,1,Viserys,unknown
8,Wilbert,Ser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,1,1,1,1,1,1,1,Wilbert,male
9,Wilbert Osgrey,Ser,NaN,NaN,NaN,NaN,NaN,House Osgrey,NaN,0,...,1,0,1,1,1,1,1,1,Wilbert,male


Decided to drop the following since where considered not to help predict if a character lived or died:
name, Mother name, Father name, culture, popularity (factor outside of the books)

other columns were also dropped given their high number of missing values and difficulty to impute them with means or modes.

In [10]:
got_c = got_c.drop(['title','culture','dateOfBirth','mother','father','heir',
                    'spouse','isAliveMother','isAliveFather','isAliveHeir',
                    'isAliveSpouse','age','first name','house','name','popularity','house'],
                    axis=1)

Decided to try different way to make bins for the gender guesser in order to understand if being male of female where big influencing factors on whether or not the character survived or if whether or not the name being common or not had influence. Ended up deciding for making bins according to sex.

In [11]:
# creating fewer bins for gender
for x in got_c['gender_guess']:
    if x == 'mostly_female':
        got_c['gender_guess'].replace(to_replace = x, value = 'female', inplace = True)
    elif x == 'mostly_male':
        got_c['gender_guess'].replace(to_replace = x, value = 'male', inplace = True)
    elif x == 'andy':
        got_c['gender_guess'].replace(to_replace = x, value = 'unknown', inplace = True)

In [12]:
# creating fewer bins for gender

#for x in got_c['gender_guess']:
#    if x== 'female' or x=='male':
#        got_c['gender_guess'].replace(to_replace = x, value = 'common', inplace = True)
#    elif x=='unknown' or x=='andy' or x =='mostly_female' or x=='mostly_male':
#        got_c['gender_guess'].replace(to_replace = x, value = 'uncommon', inplace = True)

In [13]:
#great_house = ["Bronn's House",'House Stark','House Arryn','House Tully','House Greyjoy',
#               'House Lannister','King of the Andals and the First Men','House Baratheon',
#               'House Martell','House Targaryen']

#for x in got_c['house']:
#    if x in great_house:
#        got_c['house'].replace(to_replace = x, value = 'great_house', inplace = True)
#    elif 'House' in x and x not in great_house:
#        got_c['house'].replace(to_replace = x, value = 'lesser_house', inplace = True)

In [14]:
# creating dummies of the gender_guess variable
got_c = pd.get_dummies(got_c)
got_c

,book1_A_Game_Of_Thrones,book2_A_Clash_Of_Kings,book3_A_Storm_Of_Swords,book4_A_Feast_For_Crows,book5_A_Dance_with_Dragons,isMarried,isNoble,numDeadRelations,isAlive,m_title,...,m_house,m_spouse,m_isAliveMother,m_isAliveFather,m_isAliveHeir,m_isAliveSpouse,m_age,gender_guess_female,gender_guess_male,gender_guess_unknown
0,0,0,0,0,0,0,0,11,0,1,...,1,1,0,0,0,1,1,0,0,1
1,1,1,1,1,1,1,1,1,1,0,...,0,0,1,1,1,0,0,0,0,1
2,0,0,0,1,0,0,1,0,1,0,...,0,1,1,1,1,1,1,0,0,1
3,0,0,0,0,0,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,0,0,0,1,0,1,1,0,1,0,...,0,0,1,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,1,1,1,1,1,0,0,0,0,1,...,0,1,1,1,1,1,1,0,0,1
1942,0,1,0,1,1,0,0,0,0,1,...,0,1,1,1,1,1,1,0,0,1
1943,1,1,1,1,1,0,0,0,1,1,...,1,1,1,1,1,1,1,0,1,0
1944,0,0,1,1,1,0,0,0,1,1,...,0,1,1,1,1,1,1,0,0,1


In [15]:
# declaring explanatory variables (x)
got_data = got_c.drop(['isMarried','isNoble','m_title','m_culture', 
                       'm_mother','m_father','m_heir','m_house','m_spouse', 
                       'm_isAliveMother','m_isAliveFather','m_isAliveHeir', 
                       'm_isAliveSpouse','m_dateOfBirth',
                       'isAlive','gender_guess_male'], axis=1)

# declaring response variable (y)
got_target = got_c.loc[ : , 'isAlive']


In [16]:
# splitting data into train and test sets with stratification
x_train, x_test, y_train, y_test = train_test_split(
            got_data,
            got_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = got_target) # preserving balance

#stratify is important to maintain the balance on the data and target variables 

In [17]:
# initiatinf the model object without hyperparameters
logreg = LogisticRegression(random_state = 219)


# fitting to the training set
logreg_fit = logreg.fit(x_train, y_train)


# predicting on the testing set
logreg_pred = logreg_fit.predict(x_test)


# scoring the results and saving them for future use
print('Training ACCURACY:', logreg_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', logreg_fit.score(x_test, y_test).round(4))
logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy

#measuring gap between scores and saving it
print('Logreg Train-Test Gap:', abs(logreg_train_score - logreg_test_score).round(4))
logreg_test_gap = abs(logreg_train_score - logreg_test_score).round(4)


# Calculating area under the curve and saving it
print('AUC:', roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))

logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

# unpacking the confusion matrix
logreg_tn, \
logreg_fp, \
logreg_fn, \
logreg_tp = confusion_matrix(y_true = y_test, y_pred = logreg_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {logreg_tn}
False Positives: {logreg_fp}
False Negatives: {logreg_fn}
True Positives : {logreg_tp}
""")

Training ACCURACY: 0.7681
Testing  ACCURACY: 0.8205
Logreg Train-Test Gap: 0.0524
AUC: 0.6697

True Negatives : 18
False Positives: 32
False Negatives: 3
True Positives : 142



In [18]:
# initiatinf the model object without hyperparameters
tree = DecisionTreeClassifier(random_state = 219, max_depth =8)


# fitting to the training set
tree_fit = tree.fit(x_train, y_train)


# predicting on the testing set
tree_pred = tree_fit.predict(x_test)

# scoring the results and saving them for future use
tree_train_score = tree_fit.score(x_train, y_train).round(4) # accuracy
tree_test_score  = tree_fit.score(x_test, y_test).round(4) # accuracy
print('Training ACCURACY:', tree_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', tree_fit.score(x_test, y_test).round(4))

#measuring gap between scores and saving it
print('Tree Train-Test Gap:', abs(tree_train_score - tree_test_score).round(4))
tree_test_gap = abs(tree_train_score - tree_test_score).round(4)


# Calculating area under the curve and saving it
print('AUC Score:', roc_auc_score(y_true  = y_test,
                                          y_score = tree_pred).round(4))

tree_auc_score   = roc_auc_score(y_true  = y_test,
                                        y_score = tree_pred).round(4) # auc

# unpacking the confusion matrix
tree_tn, \
tree_fp, \
tree_fn, \
tree_tp = confusion_matrix(y_true = y_test, y_pred = tree_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {tree_tn}
False Positives: {tree_fp}
False Negatives: {tree_fn}
True Positives : {tree_tp}
""")

Training ACCURACY: 0.8167
Testing  ACCURACY: 0.8359
Tree Train-Test Gap: 0.0192
AUC Score: 0.7128

True Negatives : 23
False Positives: 27
False Negatives: 5
True Positives : 140



In [19]:
# initiatinf the model object without hyperparameters
knn = KNeighborsClassifier()

# fitting to the training set
knn_fit = knn.fit(x_train, y_train)


# predicting on the testing set
knn_pred = knn_fit.predict(x_test)


# scoring the results and saving them for future use
print('KNN ACCURACY:', knn_fit.score(x_train, y_train).round(4))
print('KNN ACCURACY:',  knn_fit.score(x_test, y_test).round(4))

knn_train_score = knn_fit.score(x_train, y_train).round(4)
knn_test_score  = knn_fit.score(x_test, y_test).round(4)


#measuring gap between scores and saving it
print('KNN Train-Test Gap:', abs(knn_train_score - knn_test_score).round(4))
knn_test_gap = abs(knn_train_score - knn_test_score).round(4)

# Calculating area under the curve and saving it
print('AUC Score:', roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4))

knn_auc_score = roc_auc_score(y_true  = y_test,
                                        y_score = knn_pred).round(4) # auc

# unpacking the confusion matrix
knn_tn, \
knn_fp, \
knn_fn, \
knn_tp = confusion_matrix(y_true = y_test, y_pred = knn_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {knn_tn}
False Positives: {knn_fp}
False Negatives: {knn_fn}
True Positives : {knn_tp}
""")

KNN ACCURACY: 0.7647
KNN ACCURACY: 0.8103
KNN Train-Test Gap: 0.0456
AUC Score: 0.7348

True Negatives : 29
False Positives: 21
False Negatives: 16
True Positives : 129



In [20]:
# initiatinf the model object without hyperparameters
rf = RandomForestClassifier(random_state     = 219, max_depth =8)

# fitting to the training set
rf_fit = rf.fit(x_train, y_train)


# predicting on the testing set
rf_pred = rf_fit.predict(x_test)


# scoring the results and saving them for future use
print('Training ACCURACY:', rf_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', rf_fit.score(x_test, y_test).round(4))

rf_train_score = rf_fit.score(x_train, y_train).round(4)
rf_test_score = rf_fit.score(x_test, y_test).round(4)

#measuring gap between scores and saving it
print('RF Train-Test Gap:', abs(rf_train_score - rf_test_score).round(4))
rf_test_gap = abs(rf_train_score - rf_test_score).round(4)


# Calculating area under the curve and saving it
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = rf_pred).round(4))
rf_auc_score = roc_auc_score(y_true  = y_test,
                                          y_score = rf_pred).round(4)

# unpacking the confusion matrix
rf_tn, \
rf_fp, \
rf_fn, \
rf_tp = confusion_matrix(y_true = y_test, y_pred = rf_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {rf_tn}
False Positives: {rf_fp}
False Negatives: {rf_fn}
True Positives : {rf_tp}
""")

Training ACCURACY: 0.819
Testing  ACCURACY: 0.8154
RF Train-Test Gap: 0.0036
AUC Score        : 0.6597

True Negatives : 17
False Positives: 33
False Negatives: 3
True Positives : 142



In [21]:
# initiatinf the model object without hyperparameters
gbm = GradientBoostingClassifier(random_state  = 219, max_depth =8)


# fitting to the training set
gbm_fit = gbm.fit(x_train, y_train)


# predicting on the testing set
gbm_pred = gbm_fit.predict(x_test)


# scoring the results and saving them for future use
print('GBM ACCURACY:', gbm_fit.score(x_train, y_train).round(4))
print('GBM ACCURACY:',  gbm_fit.score(x_test, y_test).round(4))

gbm_train_score = gbm_fit.score(x_train, y_train).round(4)
gbm_test_score  = gbm_fit.score(x_test, y_test).round(4)


#measuring gap between scores and saving it
print('GBM Train-Test Gap:', abs(gbm_train_score - gbm_test_score).round(4))
knn_test_gap = abs(gbm_train_score - gbm_test_score).round(4)

# Calculating area under the curve and saving it
print('AUC Score:', roc_auc_score(y_true  = y_test,
                                          y_score = gbm_pred).round(4))

gbm_auc_score = roc_auc_score(y_true  = y_test,
                                        y_score = gbm_pred).round(4) # auc

# unpacking the confusion matrix
gbm_tn, \
gbm_fp, \
gbm_fn, \
gbm_tp = confusion_matrix(y_true = y_test, y_pred = gbm_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {gbm_tn}
False Positives: {gbm_fp}
False Negatives: {gbm_fn}
True Positives : {gbm_tp}
""")

GBM ACCURACY: 0.8207
GBM ACCURACY: 0.8308
GBM Train-Test Gap: 0.0101
AUC Score: 0.6962

True Negatives : 21
False Positives: 29
False Negatives: 4
True Positives : 141



In [22]:
# comparing results
print(f"""    
               VALUES FOR CLASSIFICATION MODELS DEFAULT VALUES
                     (except for random_state = 219, max_depth = 8)
               
Model     AUC Score     TN, FP, FN, TP     Train Score     Test Score
-----     ---------     --------------     -----------     ----------
Logistic  {logreg_auc_score}       {logreg_tn, logreg_fp, logreg_fn, logreg_tp}     {logreg_train_score}         {logreg_test_score}
Tree      {tree_auc_score}       {tree_tn, tree_fp, tree_fn, tree_tp}    {tree_train_score}         {tree_test_score}
KNN       {knn_auc_score}       {knn_tn, knn_fp, knn_fn, knn_tp}     {knn_train_score}         {knn_test_score}
Rnd Forst {rf_auc_score}       {rf_tn, rf_fp, rf_fn, rf_tp}     {rf_train_score}         {rf_test_score}
GBM       {gbm_auc_score}       {gbm_tn, gbm_fp, gbm_fn, gbm_tp}     {gbm_train_score}         {gbm_test_score}
""")


    
               VALUES FOR CLASSIFICATION MODELS DEFAULT VALUES
                     (except for random_state = 219, max_depth = 8)
               
Model     AUC Score     TN, FP, FN, TP     Train Score     Test Score
-----     ---------     --------------     -----------     ----------
Logistic  0.6697       (18, 32, 3, 142)     0.7681         0.8205
Tree      0.7128       (23, 27, 5, 140)    0.8167         0.8359
KNN       0.7348       (29, 21, 16, 129)     0.7647         0.8103
Rnd Forst 0.6597       (17, 33, 3, 142)     0.819         0.8154
GBM       0.6962       (21, 29, 4, 141)     0.8207         0.8308



Research of parameters to understand how to tune:

min_samples_split: if lower number is chosen may end up in over-fitting or if high number is chosen may end in under-fitting. 

min_samples_leaf: Helps with over-fitting if number is lower. Always ensured no matter the minimum_samples_split. 

max_features: sqrt is a good start, setting a int number could be second best option (30 to 40 percent of total number of features). can improve model performance, increases process time. 

warm_start: can help improve processing time if set to true.

splitting: leave best as default to ensure you are using the best features for the split.

Order of parameter tuning matters, some claim this might be a good order:
1) Set max depth and min_samples_split
2) min_samples leaf
3) max_features

In [23]:
# declaring a hyperparameter space
#C_range          = np.arange(0.5, 1.0, 0.1)
#warm_start_range = [True, False]
#solver_range     = ['newton-cg', 'sag', 'lbfgs']


# creating a hyperparameter grid
#param_grid = {'C'          : C_range,
#              'warm_start' : warm_start_range,
#              'solver'     : solver_range}


# initiating the model object without hyperparameters
#lr_grid = LogisticRegression(random_state = 219)


# Doing a GridSearchCV object
#lr_cv = RandomizedSearchCV(estimator               = lr_grid,
#                               param_distributions = param_grid,
#                               cv         = 10,
#                               n_iter     = 1000,
#                               scoring    = make_scorer(roc_auc_score,
#                                            needs_threshold = False))


#  fitting to the full dataset, because of cross-validation step
#lr_cv.fit(got_data, got_target)


# printing the optimal parameters and best score
#print("Tuned Parameters  :", lr_cv.best_params_)
#print("Tuned Training AUC:", lr_cv.best_score_.round(4))

In [24]:
# initiating model with best hyperparameters
lr_tuned = LogisticRegression(C = 0.7999999999999999,
                              warm_start = True,
                              solver = 'lbfgs')


#  fitting to the full dataset, because of cross-validation step
lr_tuned_fit = lr_tuned.fit(got_data, got_target)


# predicting on the testing set
lr_tuned_pred = lr_tuned_fit.predict(x_test)


# scoring the results and saving them for future use
print('LogReg Tuned Training ACCURACY:', lr_tuned.score(x_train, y_train).round(4))
print('LogReg Tuned Testing  ACCURACY:', lr_tuned.score(x_test, y_test).round(4))
lr_tuned_train_score = lr_tuned.score(x_train, y_train).round(4)
lr_tuned_test_score  = lr_tuned.score(x_test, y_test).round(4)

#calculating area under the curve and saving it
print('LogReg Tuned AUC Score :', roc_auc_score(y_true  = y_test,
                                                y_score = lr_tuned_pred).round(4))
lr_tuned_auc_score = roc_auc_score(y_true  = y_test,
                                   y_score = lr_tuned_pred).round(4) # auc

# unpacking the confusion matrix
lr_tuned_tn, \
lr_tuned_fp, \
lr_tuned_fn, \
lr_tuned_tp = confusion_matrix(y_true = y_test, y_pred = lr_tuned_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {lr_tuned_tn}
False Positives: {lr_tuned_fp}
False Negatives: {lr_tuned_fn}
True Positives : {lr_tuned_tp}
""")

LogReg Tuned Training ACCURACY: 0.7858
LogReg Tuned Testing  ACCURACY: 0.8462
LogReg Tuned AUC Score : 0.7197

True Negatives : 23
False Positives: 27
False Negatives: 3
True Positives : 142



In [25]:
# declaring a hyperparameter space
#criterion_range              = ['gini','entropy']
#splitter_range               = ['best','random']
#max_depth_range              = np.arange(1,8,1)
#min_samples_leaf_range       = np.arange(25, 50, 1)


# creating a hyperparameter grid
#param_grid = {'criterion'          :criterion_range,
#              'splitter'           :splitter_range,
#              'max_depth'          :max_depth_range,
#              'min_samples_leaf'   :min_samples_leaf_range}


# initiating the model object without hyperparameters
#tree_grid = DecisionTreeClassifier(random_state = 219)


# Doing a GridSearchCV object
#tree_cv = RandomizedSearchCV(estimator             = tree_grid,
#                               param_distributions = param_grid,
#                               cv         = 10,
#                               n_iter     = 1000,
#                               scoring    = make_scorer(roc_auc_score,
#                                            needs_threshold = False))


#  fitting to the full dataset, because of cross-validation step
#tree_cv.fit(got_data, got_target)


# printing the optimal parameters and best score
#print("Tuned Parameters  :", tree_cv.best_params_)
#print("Tuned Training AUC:", tree_cv.best_score_.round(4))

In [26]:
# initiating model with best hyperparameters
tree_tuned = DecisionTreeClassifier(criterion = 'gini',
                                    min_samples_leaf = 25,
                                    max_depth = 6,
                                    splitter = 'best',
                                    random_state = 219)


#  fitting to the full dataset, because of cross-validation step
tree_tuned_fit = tree_tuned.fit(got_data, got_target)


# predicting on the testing set
tree_tuned_pred = tree_tuned_fit.predict(x_test)


# scoring the results and saving them for future use
print('Forest Tuned Training ACCURACY:', tree_tuned.score(x_train, y_train).round(4))
print('Forest Tuned Testing  ACCURACY:', tree_tuned.score(x_test, y_test).round(4))
tree_tuned_train_score = tree_tuned.score(x_train, y_train).round(4)
tree_tuned_test_score  = tree_tuned.score(x_test, y_test).round(4)

#calculating area under the curve and saving it
print('Forest Tuned AUC Score        :', roc_auc_score(y_true  = y_test,
                                                       y_score = tree_tuned_pred).round(4))

tree_tuned_auc_score = roc_auc_score(y_true  = y_test,
                               y_score = tree_tuned_pred).round(4) # auc

# unpacking the confusion matrix
tree_tuned_tn, \
tree_tuned_fp, \
tree_tuned_fn, \
tree_tuned_tp = confusion_matrix(y_true = y_test, y_pred = tree_tuned_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {tree_tuned_tn}
False Positives: {tree_tuned_fp}
False Negatives: {tree_tuned_fn}
True Positives : {tree_tuned_tp}
""")

Forest Tuned Training ACCURACY: 0.791
Forest Tuned Testing  ACCURACY: 0.841
Forest Tuned AUC Score        : 0.7097

True Negatives : 22
False Positives: 28
False Negatives: 3
True Positives : 142



In [27]:
# declaring a hyperparameter space
#max_depth_range              = np.arange(1,8,1)
#n_estimators_range  = np.arange(100, 1000, 250)
#min_samples_leaf_range       = np.arange(25, 55, 10)
#criterion_range  = ['gini', 'entropy']
#bootstrap_range  = [True, False]
#warm_start_range = [True, False]


# creating a hyperparameter grid
#param_grid = {'max_depth'        : max_depth_range,
#              'n_estimators'     : n_estimators_range,
#              'min_samples_leaf' : min_samples_leaf_range,
#              'criterion'        : criterion_range,
#              'bootstrap'        : bootstrap_range,
#              'warm_start'       : warm_start_range}


# INSTANTIATING the model object without hyperparameters
#rf_grid = RandomForestClassifier(random_state = 219)


# GridSearchCV object
#rf_cv = RandomizedSearchCV(estimator               = rf_grid,
#                               param_distributions = param_grid,
#                               cv         = 10,
#                               n_iter     = 1000,
#                               scoring    = make_scorer(roc_auc_score,
#                                            needs_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
#rf_cv.fit(got_data, got_target)


# printing the optimal parameters and best score
#print("Tuned Parameters  :", rf_cv.best_params_)
#print("Tuned Training AUC:", rf_cv.best_score_.round(4))

In [28]:
# initiating model with best hyperparameters
rf_tuned = RandomForestClassifier(max_depth = 7,
                                  warm_start = True,
                                  n_estimators = 850,
                                  min_samples_leaf = 25,
                                  criterion = 'entropy',
                                  bootstrap = False,
                                  random_state = 219)


#  fitting to the full dataset, because of cross-validation step
rf_tuned_fit = rf_tuned.fit(got_data, got_target)


# predicting on the testing set
rf_tuned_pred = rf_tuned_fit.predict(x_test)


# scoring the results and saving them for future use
print('Forest Tuned Training ACCURACY:', rf_tuned.score(x_train, y_train).round(4))
print('Forest Tuned Testing  ACCURACY:', rf_tuned.score(x_test, y_test).round(4))
rf_tuned_train_score = rf_tuned.score(x_train, y_train).round(4)
rf_tuned_test_score  = rf_tuned.score(x_test, y_test).round(4)

#calculating area under the curve and saving it
print('Forest Tuned AUC Score:', roc_auc_score(y_true  = y_test,
                                               y_score = rf_tuned_pred).round(4))

rf_tuned_auc_score = roc_auc_score(y_true  = y_test,
                             y_score = rf_tuned_pred).round(4) # auc

# unpacking the confusion matrix
rf_tuned_tn, \
rf_tuned_fp, \
rf_tuned_fn, \
rf_tuned_tp = confusion_matrix(y_true = y_test, y_pred = rf_tuned_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {rf_tuned_tn}
False Positives: {rf_tuned_fp}
False Negatives: {rf_tuned_fn}
True Positives : {rf_tuned_tp}
""")

Forest Tuned Training ACCURACY: 0.7893
Forest Tuned Testing  ACCURACY: 0.8513
Forest Tuned AUC Score: 0.7166

True Negatives : 22
False Positives: 28
False Negatives: 1
True Positives : 144



In [29]:
#opt_neighbors = optimal_neighbors(x_data        = got_data,
#                                  y_data        = got_target,
#                                  response_type = 'class')

In [30]:
# INSTANTIATING StandardScaler()
scaler = StandardScaler()


# FITTING the data
scaler.fit(got_data)


# TRANSFORMING the data
x_scaled = scaler.transform(got_data)


# converting to a DataFrame
x_scaled_df  = pd.DataFrame(x_scaled) 


# train-test split with the scaled data
x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
            x_scaled_df,
            got_target,
            random_state = 219,
            test_size    = 0.10,
            stratify     = got_target)


# INSTANTIATING a KNN classification model with optimal neighbors
knn_opt = KNeighborsClassifier(n_neighbors = 5)


# FITTING the training data
knn_opt_fit = knn_opt.fit(x_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
knn_opt_pred = knn_opt_fit.predict(x_test_scaled)


# SCORING the results
print('Training ACCURACY:', knn_opt_fit.score(x_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', knn_opt_fit.score(x_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = knn_opt_pred).round(4))


# saving scoring data
knn_opt_train_score = knn_opt_fit.score(x_train_scaled, y_train_scaled).round(4)
knn_opt_test_score  = knn_opt_fit.score(x_test_scaled, y_test_scaled).round(4)


# saving AUC score
knn_opt_auc_score   = roc_auc_score(y_true  = y_test,
                                          y_score = knn_opt_pred).round(4)

# unpacking the confusion matrix
knn_opt_tn, \
knn_opt_fp, \
knn_opt_fn, \
knn_opt_tp = confusion_matrix(y_true = y_test, y_pred = knn_opt_pred).ravel()


# visualizing results of the confusion matrix
print(f"""
True Negatives : {knn_opt_tn}
False Positives: {knn_opt_fp}
False Negatives: {knn_opt_fn}
True Positives : {knn_opt_tp}
""")

Training ACCURACY: 0.7539
Testing  ACCURACY: 0.759
AUC Score        : 0.6741

True Negatives : 25
False Positives: 25
False Negatives: 22
True Positives : 123



In [31]:
# comparing results
print(f"""    
                      SCORES FOR CLASSIFICATION MODELS

Default Values 

Model     AUC Score     TN, FP, FN, TP     Train Score     Test Score
-----     ---------     --------------     -----------     ----------
Logistic    {logreg_auc_score}       {logreg_tn, logreg_fp, logreg_fn, logreg_tp}     {logreg_train_score}         {logreg_test_score}
Tree        {tree_auc_score}       {tree_tn, tree_fp, tree_fn, tree_tp}     {tree_train_score}         {tree_test_score}
*KNN        {knn_auc_score}       {knn_tn, knn_fp, knn_fn, knn_tp}    {knn_train_score}         {knn_test_score}
Rnd Forst   {rf_auc_score}       {rf_tn, rf_fp, rf_fn, rf_tp}     {rf_train_score}          {rf_test_score}
GBM         {gbm_auc_score}       {gbm_tn, gbm_fp, gbm_fn, gbm_tp}     {gbm_train_score}         {gbm_test_score}

Hyperparameter tuning Best
Logistic    {lr_tuned_auc_score}       {lr_tuned_tn, lr_tuned_fp, lr_tuned_fn, lr_tuned_tp}     {lr_tuned_train_score}         {lr_tuned_test_score}
Tree        {tree_tuned_auc_score}       {tree_tuned_tn, tree_tuned_fp, tree_tuned_fn, tree_tuned_tp}     {tree_tuned_train_score}          {tree_tuned_test_score}
KNN         {knn_opt_auc_score}       {knn_opt_tn, knn_opt_fp, knn_opt_fn, knn_opt_tp}    {knn_opt_train_score}         {knn_opt_test_score}
Rnd Forst   {rf_tuned_auc_score}       {rf_tuned_tn, rf_tuned_fp, rf_tuned_fn, rf_tuned_tp}     {rf_tuned_train_score}         {rf_tuned_test_score}


*BEST MODEL 
""")



    
                      SCORES FOR CLASSIFICATION MODELS

Default Values 

Model     AUC Score     TN, FP, FN, TP     Train Score     Test Score
-----     ---------     --------------     -----------     ----------
Logistic    0.6697       (18, 32, 3, 142)     0.7681         0.8205
Tree        0.7128       (23, 27, 5, 140)     0.8167         0.8359
*KNN        0.7348       (29, 21, 16, 129)    0.7647         0.8103
Rnd Forst   0.6597       (17, 33, 3, 142)     0.819          0.8154
GBM         0.6962       (21, 29, 4, 141)     0.8207         0.8308

Hyperparameter tuning Best
Logistic    0.7197       (23, 27, 3, 142)     0.7858         0.8462
Tree        0.7097       (22, 28, 3, 142)     0.791          0.841
KNN         0.6741       (25, 25, 22, 123)    0.7539         0.759
Rnd Forst   0.7166       (22, 28, 1, 144)     0.7893         0.8513


*BEST MODEL 



references for parameter tuning: 
https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
https://www.mathworks.com/help/stats/improving-classification-trees-and-regression-trees.html
https://towardsdatascience.com/how-to-tune-a-decision-tree-f03721801680
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

templates for code from class where used in this assignment